In [1]:
#from torchvision.references.detection import engine

In [1]:
import torch
import torchvision
from carpk import Carpk
from torch.utils.data import DataLoader
from torchvision import transforms
import time
from engine import train_one_epoch
import utils

In [2]:
carpklotdataset = Carpk('D:\ISO\datasets\CARPK_devkit', 'train', transform=transforms.Compose(
    [transforms.ToTensor()]))

In [3]:
data_loader=DataLoader(carpklotdataset,batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

In [4]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(num_classes=2,pretrained=False)

In [5]:
torch.cuda.is_available()

False

In [6]:
device = torch.device("cpu")

In [7]:
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform()
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(original_name=FrozenBatchNorm2d)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(original_name=FrozenBatchNorm2d)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(original_name=FrozenBatchNorm2d)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(original_name=FrozenBatchNorm2d)
          (relu): ReLU(inplace=True)
          (downsample): Sequential(
       

In [8]:
params = [p for p in model.parameters() if p.requires_grad]

In [9]:
 optimizer = torch.optim.SGD(params, lr=0.02, momentum=0.9, weight_decay=1e-4)

In [10]:
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[8,11], gamma=0.1)


In [11]:
    # Training
    print('Start training')
    start_time = time.time()
    for epoch in range(2):
        train_one_epoch(model, optimizer, data_loader, device, epoch, 1)
        lr_scheduler.step()
        if args.output_dir:
            utils.save_on_master({
                'model': model_without_ddp.state_dict(),
                'optimizer': optimizer.state_dict(),
                'lr_scheduler': lr_scheduler.state_dict(),
                'args': args},
                os.path.join(args.output_dir, 'model_{}.pth'.format(epoch)))

        # evaluate after every epoch
        evaluate(model, data_loader_test, device=device)

    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print('Training time {}'.format(total_time_str))

Start training
Epoch: [0]  [  0/495]  eta: 17:42:00  lr: 0.000060  loss: 1.8830 (1.8830)  loss_classifier: 0.7183 (0.7183)  loss_box_reg: 0.2034 (0.2034)  loss_objectness: 0.6937 (0.6937)  loss_rpn_box_reg: 0.2675 (0.2675)  time: 128.7280  data: 5.4199
Epoch: [0]  [  1/495]  eta: 19:09:19  lr: 0.000101  loss: 1.8318 (1.8574)  loss_classifier: 0.7183 (0.7203)  loss_box_reg: 0.1963 (0.1998)  loss_objectness: 0.6924 (0.6930)  loss_rpn_box_reg: 0.2209 (0.2442)  time: 139.5934  data: 2.7135


KeyboardInterrupt: 